In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json
import random


In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first

for Model in [BracketModel, TournamentModel, MatchModel, RoundModel, UserModel]:
    purge(Model)
db.session.commit()

In [6]:
# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)

# print empty user table
df

,id,role,username,email,password_hash,about_me,last_seen


In [7]:
# register user like
# curl -i -X POST -H "Content-Type: application/json" -d '{"username":"miguel","password":"python"}' http://127.0.0.1:5000/api/users

In [8]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [10]:
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)

# print empty user table
df


,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$rFTCCNhQ$5dc79d7a95633df4...,None,2020-03-07 22:51:46.023055


In [11]:
r.headers['location']

'http://127.0.0.1:5000/api/user/1'

In [12]:
# post the rest of the users
users = [
    ('TPN', 'TPN@example.com'),
    ('AngryFalco', 'AngryFalco@example.com'),
    ('Sunrisebanana', 'Sunrisebanana@example.com'),
    ('Ptolemy', 'Ptolemy@example.com'),
    ('Vik', 'Vik@example.com'),
    ('Kevin', 'Kevin@example.com'),
    ('Spaceghost', 'Spaceghost@example.com'),
    ('Burnaby', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })

    r = requests.post(url, data=payload, headers=headers)
#     print(vars(r))

In [13]:
# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$rFTCCNhQ$5dc79d7a95633df4...,None,2020-03-07 22:51:46.023055
1,2,User,TPN,tpn@example.com,pbkdf2:sha256:150000$skYqzmwd$4b29ca99e1f23f4a...,None,2020-03-07 22:51:58.545127
2,3,User,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$4DXBeW2s$8cf4a1275643d57b...,None,2020-03-07 22:51:58.645658
3,4,User,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$UkEFaBoJ$5ba1ec2cca9e0bb3...,None,2020-03-07 22:51:58.726838
4,5,User,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$noVkElzW$b5eb27fcb82d9749...,None,2020-03-07 22:51:58.808155
5,6,User,Vik,vik@example.com,pbkdf2:sha256:150000$YMPd79jW$4d0cf7bb2d7960ec...,None,2020-03-07 22:51:58.891032
6,7,User,Kevin,kevin@example.com,pbkdf2:sha256:150000$pcFr1PPO$d604112b311e630b...,None,2020-03-07 22:51:58.972022
7,8,User,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$fOD20q4U$165e82dcf1a1f10e...,None,2020-03-07 22:51:59.052929
8,9,User,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$bJkz8Iol$3d02432867f98f11...,None,2020-03-07 22:51:59.133397


In [21]:
# whenever a user registers, the response contains the url to that user's homepage
r.headers['location']

'http://127.0.0.1:5000/api/user/9'

In [14]:
# login with miguel credentials
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/login

url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=('miguel', 'python'))

In [15]:
# or get a token
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/token

In [16]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4MzYyMTUyNywiZXhwIjoxNTgzNjIyMTI3fQ.eyJpZCI6MX0.FaCgAzKq3rX6itd-LOZtzVPuefKZDwHZ11S5sEBHHUtKI_F4VfB_RrzDi9do6KyqgKnwP9_lr_5inBsbM_fknQ'

In [17]:
# now this user can login with a token
url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=(tkn, 'unused'))

In [18]:
vars(r)

{'_content': b'{"data":"Hello, miguel!"}\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Content-Type': 'application/json', 'Content-Length': '26', 'Server': 'Werkzeug/1.0.0 Python/3.6.9', 'Date': 'Sat, 07 Mar 2020 22:52:09 GMT'},
 'raw': <urllib3.response.HTTPResponse at 0x7f8305248358>,
 'url': 'http://127.0.0.1:5000/api/login',
 'encoding': None,
 'history': [],
 'reason': 'OK',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(0, 0, 10234),
 'request': <PreparedRequest [GET]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7f830527ad68>}

In [19]:
# now this user will create a tournament

In [20]:
# Fall Charity LAN 2018 Melee Singles users
N_COMPETITORS = 8
seeds = [i+1 for i in range(N_COMPETITORS)]
usernames = [
    'TPN',
    'AngryFalco',
    'Sunrisebanana',
    'Ptolemy',
    'Vik',
    'Kevin',
    'Spaceghost',
    'Burnaby',
]
np.random.shuffle(usernames)
np.random.shuffle(seeds)

In [30]:
tuple_list = [(usernames[i], seeds[i]) for i in range(N_COMPETITORS)]

In [31]:
# this our bracket (input to bracket creation):
tuple_list

[('Spaceghost', 8),
 ('Kevin', 7),
 ('TPN', 3),
 ('Vik', 4),
 ('Burnaby', 1),
 ('Ptolemy', 2),
 ('AngryFalco', 5),
 ('Sunrisebanana', 6)]

In [32]:
# create a json payload from the tuple list
url = 'http://127.0.0.1:5000/api/created-tournaments/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "users":usernames,
    "seeds":seeds,
    "tournament_name":"Fall Charity LAN 2018 Melee Singles"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
r

<Response [200]>

In [33]:
q = \
"""
    select * from match where user_1 and user_2
"""

In [34]:
df = pd.read_sql_query(q, cnx)
df

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,08a9a459-f336-42f2-9ab4-a91ff805c8e3,9,None,8,None,None,1,5,10
1,2,e9b60ce7-65e0-475f-8f81-0f5d043231ab,5,None,7,None,None,1,5,10
2,3,73fe651c-b262-4726-96e8-9acaeb47bb54,2,None,4,None,None,1,6,11
3,4,f40381c2-620d-44d5-ad32-380577f2ee49,6,None,3,None,None,1,6,11


In [29]:
active_match_ids = df.id.values
active_match_ids

array([1, 2, 3, 4])

In [48]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4MzYxODE1MCwiZXhwIjoxNTgzNjE4NzUwfQ.eyJpZCI6MX0.1aF8jDKEHj8zGMQUn81ZYXrTtIJNSDuzQAMQXp_iZqfS7ZsHlHBFKws2B9Zhu6ZJptcurK-rqMsteHaeps74zg'

In [43]:
r = requests.get(url, auth=('TPN', 'tpn'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4MzYxODA5NiwiZXhwIjoxNTgzNjE4Njk2fQ.eyJpZCI6Mn0.iDeBE3qY-Ejwo8NK3Sz_PPOaj0gzxfA01RG3S1T1wvUcjRwEc6dae8ycCx5tn9xCaf4DmLlwLh_d6djeHLmF-A'

In [44]:
payload = json.dumps({
        "entrant1_score" : 3,
        "entrant2_score" : 2,
        "winner" : 9,
        "loser" : 8
})

In [49]:
url = f'http://127.0.0.1:5000/api/match/1/report-match'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
r.content

b'{"id":1,"loser_to":10,"match_winner":9,"round":1,"u1":9,"u2":8,"user_1_score":3,"user_2_score":2,"uuid":"08a9a459-f336-42f2-9ab4-a91ff805c8e3","winner_to":5}\n'

In [31]:
for i in active_match_ids:
    user_ids = [
        df.loc[df['id'] == i].user_1.values[0],
        df.loc[df['id'] == i].user_2.values[0]
    ]
    scores = random.sample(range(5), 2)
    w_id = user_ids[np.argmax(scores)]
    l_id = user_ids[np.argmin(scores)]
#     print(w_id)
#     print(l_id)
    payload = json.dumps({
        "entrant1_score" : str(scores[0]),
        "entrant2_score" : str(scores[1]),
        "winner" : str(w_id),
        "loser" : str(l_id)
    })
    url = f'http://127.0.0.1:5000/api/match/{i}/report-match'
    headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
    r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
    print(r.content)
    break
    

b'{"id":1,"loser_to":10,"match_winner":4,"round":1,"u1":4,"u2":5,"user_1_score":4,"user_2_score":3,"uuid":"7106b296-5b66-4d64-bfe6-76c8d9ae5fee","winner_to":5}\n'


In [ ]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "entrant1_score":4,
    "entrant2_score":3,
    "winner":,
    "loser":,
})

In [11]:

TO = t.bracket.entrants[0]

# initial post to db
t.post_to_db(tournament_name, TO,)

In [12]:
# post self references in matches separately
for r in t.bracket.rounds:
    for m in r.matches:
        m.post_self_refs()

In [13]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,TPN,tpn@example.com,None,None,2020-02-20 01:20:58.042862
1,2,AngryFalco,angryfalco@example.com,None,None,2020-02-20 01:20:58.043367
2,3,Sunrisebanana,sunrisebanana@example.com,None,None,2020-02-20 01:20:58.043462
3,4,Ptolemy,ptolemy@example.com,None,None,2020-02-20 01:20:58.043541
4,5,Vik,vik@example.com,None,None,2020-02-20 01:20:58.043612
5,6,Kevin,kevin@example.com,None,None,2020-02-20 01:20:58.043696
6,7,Spaceghost,spaceghost@example.com,None,None,2020-02-20 01:20:58.043772
7,8,Burnaby,burnaby@example.com,None,None,2020-02-20 01:20:58.043859


In [14]:
pd.read_sql_query('select * from tournament', cnx)

,id,n_entrants,name,organizer_id
0,1,8,Fall Charity LAN 2018 Melee Singles,2


In [15]:
pd.read_sql_query('select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [16]:
pd.read_sql_query('select * from bracket_entrants', cnx)

,user_id,bracket_id
0,2,1
1,8,1
2,5,1
3,3,1
4,6,1
5,7,1
6,1,1
7,4,1


In [17]:
pd.read_sql_query('select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1
1,2,2,1,1
2,3,3,1,1
3,4,4,1,1
4,5,5,1,1
5,6,1,0,1
6,7,2,0,1
7,8,3,0,1
8,9,4,0,1


In [18]:
df = pd.read_sql_query('select * from match', cnx)
df
# df.loc[df['uuid'] == '94de92a2-13ac-4e9a-9bf2-78636100a7b1']

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,0f9a1604-97d0-4a42-b510-56f7ac0f5ae6,2.0,None,4.0,None,None,1,5.0,10.0
1,2,f4f7edfb-9d3b-4348-88a6-cf4dc2eb7cf2,8.0,None,1.0,None,None,1,5.0,10.0
2,3,e3f3fe1d-4907-45e0-a70e-57b32aebec1e,5.0,None,7.0,None,None,1,6.0,11.0
3,4,d2785cd6-9819-4005-8404-b3cd4059e164,3.0,None,6.0,None,None,1,6.0,11.0
4,5,f31792a1-2cc5-4b81-9771-3f0d539c0734,NaN,None,NaN,None,None,2,7.0,12.0
5,6,f8cf1b92-7bb5-4a74-b88d-7e9dd78217f5,NaN,None,NaN,None,None,2,7.0,13.0
6,7,ec8fb7c6-f4bd-4366-a720-d1470479b824,NaN,None,NaN,None,None,3,8.0,15.0
7,8,80667cdb-737f-45e4-ad90-e9cf472506df,NaN,None,NaN,None,None,4,NaN,NaN
8,9,e2e48a25-f7e9-4ada-9dc0-b6a1e889aa36,NaN,None,NaN,None,None,5,NaN,NaN
9,10,e3f465e6-eb1a-4e62-9537-dfa1aaf32132,NaN,None,NaN,None,None,6,12.0,NaN
